In [ ]:
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
ds = load_dataset("mlburnham/dem_rep_party_platform_topics")
df = concatenate_datasets(ds.values()).to_pandas()
df

Dropping useless columns.

In [ ]:
df = df.drop(columns=["target", "hypothesis", "entailment", "validation_label", "validation_source"])

Renaming columns.

In [ ]:
df = df.rename(columns={"premise": "body", "dataset": "leaning"})

Categorizing the leaning column.

In [ ]:
df["leaning"].unique()

In [ ]:
df["leaning"] = df["leaning"].replace({"Dem. Party Platforms": "left", "Rep. Party Platforms": "right"})
df["leaning"] = df["leaning"].astype("category")

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Inspecting body length.

In [ ]:
df["body_length"] = df["body"].str.len()
df["body_word_count"] = df["body"].str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
df["body_length"].mean()

The leaning distribution.

In [ ]:
df.groupby("leaning", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per leaning.

In [ ]:
df.groupby("leaning", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

Body length distribution by leaning.

In [ ]:
plt.figure(figsize=(10, 6))

for leaning in df["leaning"].unique():
    df_leaning = df[df["leaning"] == leaning]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_leaning["body_length"].iloc[::100], df_leaning["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=leaning)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by political leaning")
plt.legend(title="leaning")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/dem_rep_party_platform_topics.parquet")

In [ ]:
df